In [7]:
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
import random
from time import sleep
from pandas import json_normalize
import pandas as pd
import pickle
import ast
from sklearn.preprocessing import StandardScaler

In [8]:
secrets_file = open("secret.txt","r")

string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['CDI'],
                                                           client_secret=secrets_dict['Secret']))

In [9]:
forrecomender = pd.read_csv('forrecomender.csv')


In [10]:
##Load kmeans model
kmeans = pickle.load(open('kmeans_model.sav', 'rb'))
##Load Scaler
scaler = pickle.load(open('standard_scaler.sav', 'rb'))

In [11]:
kmeans

KMeans(n_clusters=11, random_state=1234)

In [12]:
songs = pd.read_csv("100songs.csv")


In [13]:
songs

,Unnamed: 0,artist_name,title
0,0,Miley Cyrus,Flowers
1,1,Morgan Wallen,Last Night
2,2,JISOO,FLOWER
3,3,Luke Combs,Fast Car
4,4,Russ,NASTY (Extended Version)
...,...,...,...
95,95,Brandon Lake,Gratitude
96,96,Katy Nichole & Big Daddy Weave,God Is In This Story
97,97,Lady Gaga,Bloody Mary
98,98,Fastball,Out of My Head


In [73]:

 
def hotsong(hotcandidate):
    
    x=0
    test=songs.sample(n = 1)
    name= test["artist_name"].to_string(index=False, header=False) 
    nametitle= " " + "-" + " " + test["title"].to_string(index=False, header=False) 
        
    for i in songs["title"]:
            if i==hotcandidate:
                test = songs[songs['title'] != hotcandidate]
                test=songs.sample(n = 1)
                test= name + nametitle
                answer="Damn thats hot!!! You might like this song:  \n " + test
                x=0
                break
            else:
                x+=1

    if x==100:
        ## asking for a sogn details ffrom spotify
        query=hotcandidate
        results = sp.search(q=query, type="track", limit=1, offset=0)
        ##getting uri
        boom=results["tracks"]["items"][0]["id"]
        ##assigning to dataframe 
        dick=sp.audio_features(boom)[0]
        dick=pd.DataFrame([dick])
        ## removing all unwanted columns 
        dick.drop(dick.columns.difference(['danceability', 'energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']), axis = 1, inplace = True)
        ##transform in order to corr with dataframe
        audio_feat_df_corr_prep = scaler.transform(dick)
        ## predict kmean of the song 
        clusters1 = kmeans.predict(audio_feat_df_corr_prep)
        ## ask for the one sample track that can be given 
        recomendedtrax=forrecomender[forrecomender['cluster'] == clusters1[0]].reset_index()
        test=recomendedtrax.sample()
        name= test["artists"].to_string(index=False, header=False) 
        nametitle= " " + "-" + " " + test["title"].to_string(index=False, header=False)
        test= name + nametitle
        answer="Your song was not found, but you might like:  \n " + test

    return answer
    
###works as a function , returns from spotify an object , still need to figure
##out how to work this another way but solution is valid 

In [78]:
hotcandidate = input("Please enter a song and we will check if its hot:")
hotsong(hotcandidate)

##final results 

Please enter a song and we will check if its hot:sun always


'Your song was not found, but you might like:  \n ["Guns N\' Roses"] - [\'Mr. Brownstone\']'